In [ ]:
from api_key import API_KEY
from datetime import datetime as dt
from random import randint, uniform

import json
import requests

In [ ]:
DB_URL = "https://pigeonblog-db-default-rtdb.firebaseio.com"
AUTH_URL = "https://identitytoolkit.googleapis.com/v1/accounts:signUp"

### Auth

In [ ]:
post_url = f"{AUTH_URL}?key={API_KEY}"
res = requests.post(post_url, json={"returnSecureToken": True})
if res.status_code == 200:
  token = json.loads(res.text)["idToken"]
  print("ok")

### Nested schema

In [ ]:
session_id = dt.now().strftime("%Y%m%d")
pigeon_id = randint(0, 4)

sensors = { f"sen{i}": round(uniform(0.0, 16.0), 4) for i in range(8) }

measurement = {
  "timestamp": {".sv": "timestamp"},
  "lat": round(uniform(33.730579, 34.334744), 4),
  "lon": round(uniform(-118.656238, -118.159107), 4),
  **sensors,
}

post_url = f"{DB_URL}/sessions/{session_id}/pigeons/{pigeon_id}/measurements.json?auth={token}"
res = requests.post(post_url, json=measurement)
print(res.text)

### Flat schema

In [ ]:
pigeon_id = randint(0, 4)

sensors = { f"sen{i}": round(uniform(0.0, 16.0), 4) for i in range(8) }

measurement = {
  "timestamp": {".sv": "timestamp"},
  "pigeon": pigeon_id,
  "lat": round(uniform(33.730579, 34.334744), 4),
  "lon": round(uniform(-118.656238, -118.159107), 4),
  **sensors,
}

post_url = f"{DB_URL}/measurements.json?auth={token}"
res = requests.post(post_url, json=measurement)
print(res.text)

### Flat A9G

In [ ]:
datapack = {
  "timestamp": {".sv": "timestamp"},
  "pigeon": randint(0, 4),
  "lat": round(uniform(33.730579, 34.334744), 4),
  "lon": round(uniform(-118.656238, -118.159107), 4),
  "sata": 0,
  "satb": 0,
  "temp": randint(10, 35),
  "pres": randint(750, 800),
  "alti": randint(10, 1000),
  "NH3": round(uniform(18.00, 22.0), 4),
  "OXI": round(uniform(18.00, 22.0), 4),
  "RED": round(uniform(18.00, 22.0), 4),
  "local": int(dt.now().timestamp()),
}

post_url = f"{DB_URL}/measurements.json?auth={token}"
res = requests.post(post_url, json=datapack)
print(res.text)

### ~2 minutes of test data

In [ ]:
from random import shuffle
from time import sleep

p_list = list(range(5))

for m in range(128):
  shuffle(p_list)
  for p in p_list:
    datapack = {
      "timestamp": {".sv": "timestamp"},
      "pigeon": p,
      "lat": round(uniform(33.730579, 34.334744), 4),
      "lon": round(uniform(-118.656238, -118.159107), 4),
      "sata": 0,
      "satb": 0,
      "temp": randint(10, 35),
      "pres": randint(750, 800),
      "alti": randint(10, 1000),
      "NH3": round(uniform(18.00, 22.0), 4),
      "OXI": round(uniform(18.00, 22.0), 4),
      "RED": round(uniform(18.00, 22.0), 4),
      "local": int(dt.now().timestamp()),
    }

    post_url = f"{DB_URL}/measurements.json?auth={token}"
    res = requests.post(post_url, json=datapack)
    sleep(uniform(0.1, 0.5))
  sleep(1.0)


### seed the /sessions/ part of the db

In [ ]:
def get_start_of_day(ts):
  return ts - (ts % (24 * 60 * 60 * 1000)) + 1000

def get_first_measurement_after(ts):
  start_of_day = get_start_of_day(ts)
  start_of_next_day_ts = start_of_day + (24 * 60 * 60 * 1000)

  get_url = f'{DB_URL}/measurements.json?orderBy="timestamp"&startAt={start_of_next_day_ts}&limitToFirst=1'
  res = requests.get(get_url)
  if res.status_code == 200:
    res_obj = json.loads(res.content)
    if res_obj is not None:
      return next(iter(res_obj.values()))
  return {}

In [ ]:
unique_days = set()

latest_measurement = {
  "timestamp": 1000
}

while "timestamp" in latest_measurement:
  latest_measurement = get_first_measurement_after(latest_measurement["timestamp"])
  if "timestamp" in latest_measurement:
    unique_days.add(get_start_of_day(latest_measurement["timestamp"]))

unique_days

In [ ]:
d = next(iter(unique_days))

for d in unique_days:
  put_url = f"{DB_URL}/sessions/{str(d)}.json?auth={token}"
  res = requests.put(put_url, data="true")
  print(res.status_code, res.content)
